## PD结合的思路

### 分别替换gate/up层

先输入一句话，按照70%稀疏去记录prefill阶段激活的神经元，最后统计这个输入prompt对应的最高激活次数的70%的神经元。

In [1]:
import os
import json
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from transformers import GenerationConfig
from datasets import load_dataset

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### from path.json read paths of model and dataset
model_name = "Llama3-8b"
dataset_name = "c4"
with open('path.json', 'r') as file:
    paths = json.load(file)
    model_path = paths.get(model_name, '')
    dataset_path = paths.get(dataset_name, '')

c4 = load_dataset(dataset_path)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    use_cache=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from tqdm import tqdm
import convert_llama
# for gamma in [0,0.2]:
#     for beta in [0,0.2]:
SAMPLE_NUM = 1
MAX_LENGTH = 300
OUTPUT_LENTGH = 2
alpha = 0.2
# for gamma,beta in [(0,0.3),(0.3,0),(0.2,0.2)]:
for gamma, beta in [(0.3,0.1)]:
# for gamma, beta in [(0,0.1)]:
# for gamma, beta in [(0.3221,0)]:
# for gamma,beta in [(0.1,0.2),(0.2,0.1),(0.2,0.2)]:
    print(f"alpha: {alpha}, gamma: {gamma}, beta: {beta}")
    convert_llama.convert_llama_model(model, sparsity=0.1, start_num=20, end_num=32, alpha=alpha, beta=beta, gamma=gamma, use_core=True)
    for layerid in range(22,32):
        model.model.layers[layerid].mlp.clear_list()

    # for c4_demo in c4['validation']['text'][:SAMPLE_NUM]:
    for c4_demo in tqdm(c4['validation']['text'][:SAMPLE_NUM]):
        input_demo = tokenizer(c4_demo, padding=False, truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
        if input_demo.input_ids.shape[1] < MAX_LENGTH:
            continue
        print(tokenizer.batch_decode(input_demo.input_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
        generate_ids = model.generate(input_demo.input_ids.to('cuda:0'), max_length=MAX_LENGTH+OUTPUT_LENTGH, generation_config=GenerationConfig(do_sample=False), pad_token_id=tokenizer.eos_token_id)
        # tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])
    for layerid in range(22,32):
        model.model.layers[layerid].mlp.coreinfer_recall()


alpha: 0.2, gamma: 0.3, beta: 0.1


Convert Llama Models: 0it [00:00, ?it/s]

Converting Layer model.layers.21.mlp with CoreInfer
Converting Layer model.layers.22.mlp with CoreInfer


Convert Llama Models: 318it [00:00, 864.53it/s]

Converting Layer model.layers.23.mlp with CoreInfer
Converting Layer model.layers.24.mlp with CoreInfer
Converting Layer model.layers.25.mlp with CoreInfer
Converting Layer model.layers.26.mlp with CoreInfer
Converting Layer model.layers.27.mlp with CoreInfer


...

Converted Model Done


  0%|          | 0/1 [00:00<?, ?it/s]

The woman who died after falling from a bridge over the A21 has been identified as a Sevenoaks mum.
Marta Kendle, 37, fell from the Gracious Lane bridge on the morning of February 19.
Police were called to the carriageway around 6.10am and the road was promptly closed in both directions.
Despite paramedics best efforts, Marta, who was originally from Poland, was pronounced dead at the scene.
Kent and Medway Coroners office have confirmed an inquest into her death will open on Wednesday (February 27).
Tributes to the mum were left at the scene and on social media.
Friend, Jodi Cahill posted on Facebook: "I will certainly remember you. I am sorry we did not see how lost and alone you felt.
"Be at peace dear Marta."
A floral tribute left at the scene said goodbye to the "beautiful and kind soul".
It read: "To a beautiful and kind soul, we are sorry we didn&apos;t see your pain. We will miss you always.
"Rest in peace Marta, Love from Lorraine, Colin and family."
The inquest will take plac

100%|██████████| 1/1 [00:00<00:00,  3.63it/s]

The woman who died after falling from a bridge over the A21 has been identified as a Sevenoaks mum.
Marta Kendle, 37, fell from the Gracious Lane bridge on the morning of February 19.
Police were called to the carriageway around 6.10am and the road was promptly closed in both directions.
Despite paramedics best efforts, Marta, who was originally from Poland, was pronounced dead at the scene.
Kent and Medway Coroners office have confirmed an inquest into her death will open on Wednesday (February 27).
Tributes to the mum were left at the scene and on social media.
Friend, Jodi Cahill posted on Facebook: "I will certainly remember you. I am sorry we did not see how lost and alone you felt.
"Be at peace dear Marta."
A floral tribute left at the scene said goodbye to the "beautiful and kind soul".
It read: "To a beautiful and kind soul, we are sorry we didn&apos;t see your pain. We will miss you always.
"Rest in peace Marta, Love from Lorraine, Colin and family."
The inquest will take plac

In [43]:
import torch
from tqdm import tqdm
import convert_llama
import numpy as np
import math
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. 定义超参数
SAMPLE_NUM = 200
MAX_LENGTH = 300
OUTPUT_LENTGH = 30
alpha = 0.2
beta = 0
gamma = 1.0

eos_token_id = tokenizer.convert_tokens_to_ids('.')

convert_llama.convert_llama_model(model, sparsity=0.1, start_num=20, end_num=32, alpha=alpha, beta=beta, gamma=gamma)
ppl_list = []
# 2. 准备输入文本
for c4_demo in tqdm(c4['validation']['text'][:SAMPLE_NUM]):
    input_demo = tokenizer(c4_demo, padding=False, truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
    if input_demo.input_ids.shape[1] < MAX_LENGTH:
        continue
    # input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model(input_demo.input_ids.cuda(), use_cache=True)
        past_key_values = outputs.past_key_values
    
    generated = input_demo.input_ids
    log_prob = []
    # 3. 生成文本[不应该用model.generate, 手写一个方便测试困惑度]
    for _ in range(OUTPUT_LENTGH):
        with torch.no_grad():
            input_ids = generated[:, -1:].cuda()
            outputs = model(input_ids=input_ids, past_key_values=past_key_values, use_cache=True)
            logits = outputs.logits
            past_key_values = outputs.past_key_values

            # print(logits.shape) # [1, 1, 128256]
            # print(input_ids.shape) # [1, 1]
            # labels = input_ids[:, :]
            probs = torch.softmax(logits[:,:], dim=-1)
            probs = torch.max(probs.squeeze(0))
            # print(probs, labels)
            # print(probs)
            log_prob.append(probs.log2().cpu().numpy())
    
        next_token_logits = logits[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1)
    
        generated = torch.cat((generated.cuda(), next_token_id.unsqueeze(-1)), dim=1)
    
        next_token_text = tokenizer.decode(next_token_id)
        # print(next_token_text, end='', flush=True)
    
        # if next_token_id.item() == eos_token_id:
        #     break
        # if '.' in next_token_text:
        #     break
    ppl_item = - np.sum(log_prob) / len(log_prob)
    ppl_list.append(ppl_item)


ce = np.sum(ppl_list) / len(ppl_list)
ppl = 2 ** ce
print(f"Perplexity: {ppl.item()}")

Convert Llama Models: 0it [00:00, ?it/s]

Convert Llama Models: 462it [00:02, 208.40it/s]


Converted Model Done


100%|██████████| 200/200 [03:20<00:00,  1.00s/it]

Perplexity: 2.3604316777980006


In [2]:
### 使用greedy decode
generate_ids = model.generate(input_demo.input_ids.to('cuda:0'), max_length=230, generation_config=GenerationConfig(do_sample=False), pad_token_id=tokenizer.eos_token_id)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

[prefill] in gate layer: 15
[prefill] in up layer: 15


'The woman who died after falling from a bridge over the A21 has been identified as a Sevenoaks mum.\nMarta Kendle, 37, fell from the Gracious Lane bridge on the morning of February 19.\nPolice were called to the carriageway around 6.10am and the road was promptly closed in both directions.\nDespite paramedics best efforts, Marta, who was originally from Poland, was pronounced dead at the scene.\nKent and Medway Coroners office have confirmed an inquest into her death will open on Wednesday (February 27).\nTributes to the mum were left at the scene and on social media.\nFriend, Jodi Cahill posted on Facebook: "I will certainly remember you. I am sorry we did not see how lost and alone you felt.\n"Be at peace dear Marta."\nA floral tribute left at the scene said goodbye to the "beautiful and kind soul".\nIt read: "To a beautiful and kind soul. You will be missed. Rest in peace."\nA spokesman for Kent Police said: "Officers were called to the A21 at Gracious Lane,'

In [3]:
model.model.layers[15].mlp.gate_proj.coreinfer_recall()
model.model.layers[15].mlp.up_proj.coreinfer_recall()

in decode, gate layer 15
Overlap count: 1243.8621, Overlap ratio: 0.8680
in decode, up layer 15
Overlap count: 1080.8966, Overlap ratio: 0.7543


## 加载保存的激活值

In [10]:
from torch import nn
import torch.nn.init as init
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast  
from torch.utils.data import DataLoader, Dataset, random_split
import torch.optim as optim
import torch
import json

with open('path.json', 'r') as file:
    paths = json.load(file)
    save_path = paths.get('gatemulup_path','')

def load_datasets(layerid = 1, expertid = 0, startid=1, endid=4, use_x1 = False):   
    datasets_x = []
    datasets_y = []
    datasets_x1 = []
    for fileid in range(startid, endid):
        # print(fileid)
        # 加一个map_location
        d = torch.load(f'{save_path}/{fileid}-{layerid}-more.pth', map_location=lambda storage, loc: storage.cuda(0))
        datasets_x.append(d[0])
        if use_x1:
            datasets_x1.append(d[1])
        datasets_y.append(d[-1])
    x,y = torch.cat(datasets_x,dim=1), torch.cat(datasets_y,dim=1)
    datasets_x.clear()
    datasets_y.clear()
    x = x.reshape(-1, 4096)
    y = y.reshape(-1, 14336)
    # print(x[0].shape)
    if use_x1:
        x1 = torch.cat(datasets_x1,dim=1)
        datasets_x1.clear()
        x1 = x1.reshape(-1, 14336)
        return x,x1,y
    return x,y
    

class CustomDataset(Dataset):
    def __init__(self, layerid = 1, expertid = 0, startid=1, endid=4, use_x1 =False):
        # 加载数据self.data_x1,
        self.use_x1 = use_x1
        if use_x1:
            self.data_x, self.data_x1, self.data_y = load_datasets(layerid,startid=startid,endid=endid,use_x1=use_x1)
            print(len(self.data_x1),len(self.data_x),len(self.data_y))
        else:
            self.data_x, self.data_y = load_datasets(layerid,startid=startid,endid=endid,use_x1=use_x1)
            print(len(self.data_x),len(self.data_y))

    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        if self.use_x1:
            return self.data_x[idx],self.data_x1[idx],self.data_y[idx]
        else:
            return self.data_x[idx],self.data_y[idx]

## 稀疏预测器

In [11]:
class SimpleLinearModel(nn.Module):
    def __init__(self,input_dim,output_dim,hidden_dim=32):
        super(SimpleLinearModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim,bias=False)
        # self.activation = nn.SiLU() # 添加激活函数
        self.linear2 = nn.Linear(hidden_dim,output_dim,bias=False)  
        init.kaiming_normal_(self.linear1.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.linear2.weight, mode='fan_out', nonlinearity='relu')
        # self.linear1.bias.data.fill_(0)
        # self.linear2.bias.data.fill_(0)

    def forward(self, x):
        # x= self.activation(x)
        return self.linear2(self.linear1(x))
    
model=SimpleLinearModel(4096,14336,hidden_dim=1024)
model.to("cuda")  # 假设使用 GPU
# criterion = nn.MSELoss().to("cuda")
criterion = nn.CrossEntropyLoss().to("cuda")
# criterion = nn.KLDivLoss(reduction='batchmean').to("cuda")
optimizer = optim.Adam(model.parameters(), lr=5e-4) #lr=5e-5
writer = SummaryWriter('runs/predictor_sparsity')

In [12]:
from tqdm import tqdm

cnt = 0

def sparse_row(row, keep_ratio=0.1, use_abs = False):
    # 计算需要保留的参数数量
    num_to_keep = int(keep_ratio * row.numel())
    
    # 找到绝对值最大的 num_to_keep 个参数的索引
    if use_abs:
        row = torch.abs(row)
    topk_indices = torch.topk(row, num_to_keep).indices
    # topk_indices = torch.topk(row, num_to_keep).indices
    
    # 创建一个与 row 相同大小的零张量
    sparse_row = torch.zeros_like(row)
    
    # 将 topk_indices 对应的值置为 1
    sparse_row[topk_indices] = 1
    
    return sparse_row

def generate_label(y, sparsity, use_abs=False):
    # 对每一行进行稀疏化
    sparse_tensor = torch.stack([sparse_row(row, sparsity, use_abs) for row in y])
    return sparse_tensor

def test_model(model, val_loader, sparsity=0.1):
    model.eval()
    # 初始化总的统计变量
    total_correct_preds = 0
    total_preds = 0
    total_labels = 0
    total_masks = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in tqdm(enumerate(val_loader)):
            with autocast():
                outputs = model(inputs)

            preds = generate_label(outputs, sparsity)
            truth = generate_label(targets, 0.1, use_abs = True)
            # truth = targets
            
            # 计算当前batch的精度
            dif = truth - preds
            miss = dif > 0.0 # classifier didn't activated target neuron

            total_correct_preds += (truth.sum(dim=1).float() - miss.sum(dim=1).float()).mean().item()
            total_preds += (preds.sum(dim=1).float()).mean().item()
            total_labels += (truth.sum(dim=1).float()).mean().item()

    # print('预测占比:{:.4f}'.format((total_preds/total_masks).item()))
    # print('标签占比:{:.4f}'.format((total_labels/total_masks).item()))
    print('预测与标签选取的数量比:',(total_preds / total_labels))
    print('覆盖率(Recall):',(total_correct_preds / total_labels))

def train_model(model, train_loader, val_loader, criterion, optimizer, writer, epochs=25, layerid=1):
    scaler = GradScaler()  # 创建 GradScaler 对象
    for epoch in range(epochs):
        if epoch % 2 == 0:
            print(f'---------after training {epoch} epochs---------')
            test_model(model, val_loader, sparsity=0.2)
        model.train()
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.cuda(), targets.cuda()

            optimizer.zero_grad()

            targets = generate_label(targets, 0.2, use_abs =True)

            # 使用 autocast 来进行自动混合精度处理
            with autocast():
                outputs = model(inputs)
                probs = outputs.sigmoid()
                # cross_entropy
                loss = criterion(probs, targets)

            # 使用 GradScaler 来缩放损失，然后进行反向传播
            # 注意：反向传播不包含在 autocast() 块中
            scaler.scale(loss).backward()
            writer.add_scalar('Loss/Train', loss.item(), epoch * len(train_loader) + batch_idx)
            # 调用 scaler.step() 来更新模型权重，并调用 scaler.update() 准备下一步
            scaler.step(optimizer)
            scaler.update()
    print(f'---------after training {epochs} epochs---------')
    test_model(model, val_loader, sparsity=0.2)
    global cnt
    torch.save(model.state_dict(), f'./output/sparsity/{layerid}-{cnt}.pt')
    cnt += 1


In [4]:
for startid, endid in [(1,4),(4,7),(7,10)]:
    layerid = 15
    dataset = CustomDataset(layerid, startid=startid, endid=endid)
    print(len(dataset), dataset[0][0].shape, dataset[0][1].shape) # torch.Size([512, 4096])
    # 划分训练集和验证集
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

    train_model(model, train_loader, val_loader, criterion, optimizer, writer=writer, epochs=4, layerid=15)

333186 333186
333186 torch.Size([4096]) torch.Size([14336])
---------after training 0 epochs---------


33it [00:08,  3.91it/s]


预测与标签选取的数量比: 2.0013956734124214
覆盖率(Recall): 0.20009789296144784
---------after training 2 epochs---------


33it [00:07,  4.16it/s]


预测与标签选取的数量比: 2.0013956734124214
覆盖率(Recall): 0.5413863739824867
---------after training 4 epochs---------


...

预测与标签选取的数量比: 2.0013956734124214
覆盖率(Recall): 0.5556886096016526
---------after training 2 epochs---------


34it [00:08,  4.22it/s]


预测与标签选取的数量比: 2.0013956734124214
覆盖率(Recall): 0.5726461708435582
---------after training 4 epochs---------


34it [00:08,  4.20it/s]


预测与标签选取的数量比: 2.0013956734124214
覆盖率(Recall): 0.5752094106414646


### 加载训练好的进行测试

In [13]:
layerid = 15
dataset = CustomDataset(layerid, startid=1, endid=4)
print(len(dataset), dataset[0][0].shape, dataset[0][1].shape) # torch.Size([512, 4096])
# 划分训练集和验证集
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
# model = SimpleLinearModel(4096,14336,hidden_dim=1024).cuda()
model.load_state_dict(torch.load(f'./output/sparsity/15-2.pt'))
test_model(model, val_loader, sparsity=0.3)

333186 333186
333186 torch.Size([4096]) torch.Size([14336])


33it [00:09,  3.30it/s]

预测与标签选取的数量比: 3.0006978367062107
覆盖率(Recall): 0.6927953512494317


### 下游任务测试

In [1]:
# Test Model
import os
import json
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from convert_llama import convert_llama_model
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from lm_eval.models.huggingface import HFLM
from lm_eval import evaluator


def _load_model(model_name = "Llama3-8b"):
    print(f"Loading model {model_name}")
    ### from path.json read paths of model and dataset
    with open('path.json', 'r') as file:
        paths = json.load(file)
        model_path = paths.get(model_name, '')

    model = LlamaForCausalLM.from_pretrained(
        model_path,
        device_map='auto',
        use_cache=True,
        torch_dtype=torch.float16,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return model, tokenizer

def evaluate(task_name, model, tokenizer, num_fewshot, device):
    hflm = HFLM(pretrained=model, tokenizer=tokenizer)
    results = evaluator.simple_evaluate(
    model=hflm,
    tasks=[task_name],
    num_fewshot=num_fewshot)
    print(results['results'])


def main(task_name, model_name, sparsity, start_num, end_num, token_sparsity, memory_limit, device, num_fewshot,):
    model, tokenizer = _load_model(model_name)
    
    model = convert_llama_model(model, sparsity, start_num, end_num, token_sparsity,)

    evaluate(task_name, model, tokenizer, num_fewshot, device)

main(task_name='boolq', model_name="Llama3-8b", sparsity=0.1, start_num=21, end_num=32, token_sparsity=0.1, memory_limit=0.1, device='cuda', num_fewshot=1)

KeyboardInterrupt: 